In [ ]:
# install libraries required
!pip install pydantic-evals   # evaluation framework
!pip install pydantic-ai      # the LLM judge interface
!pip install openai           # used as the judge model


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.7/429.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 141.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38

In [ ]:
# load libraries
import json
import os
# from google import genai
from google.colab import userdata
from pydantic import BaseModel, Field
from pydantic_evals.evaluators import Evaluator, EvaluationResult
from pydantic_ai import Agent


In [ ]:
# import pydantic_evals.evaluators
# print(dir(pydantic_evals.evaluators))

In [ ]:
import pydantic_evals
print(dir(pydantic_evals))

['Case', 'Dataset', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_utils', 'dataset', 'evaluators', 'increment_eval_metric', 'otel', 'reporting', 'set_eval_attribute']


In [ ]:
# load input files

# load rubric, submission and prompt
with open("rub_it_0002.json", "r") as f:
    rubric_data = json.load(f)

# load llm feedback
with open("FG_Feedback_Results_with_Scores.json", "r") as f:
    feedback_data = json.load(f)


In [ ]:
# extract rubric, submission, and feedback
rubric = rubric_data["rubric"]["criteria"]
student_submission = rubric_data["submissions"][0]["final_submission"]
llm_feedback = feedback_data[0]["feedback"]


In [ ]:
print("Rubric: ", rubric, "\n")
print("Submission: ", student_submission, "\n")
print("Feedback: ", llm_feedback, "\n")

Rubric:  [{'criterion_id': 'c1', 'name': 'Conceptual Understanding', 'description': 'Demonstrates deep understanding of the topic and related concepts.', 'performance_descriptors': {'excellent': 'Shows precise and in-depth understanding with clear theoretical support.', 'good': 'Understands key concepts well with minor gaps.', 'average': 'Adequate explanation with some inaccuracies.', 'needs_improvement': 'Surface-level or confused understanding.', 'poor': 'Major misunderstandings or misrepresentations.'}, 'weight': 15}, {'criterion_id': 'c2', 'name': 'Application to Real-World Scenarios', 'description': 'Applies concepts meaningfully to examples or scenarios.', 'performance_descriptors': {'excellent': 'Uses insightful and relevant examples clearly linked to concepts.', 'good': 'Examples mostly appropriate with some gaps in explanation.', 'average': 'Examples present but superficial or unclear connections.', 'needs_improvement': 'Vague or generic examples.', 'poor': 'No real-world appl

In [ ]:
# define the meta-rubric for the llm judge

META_RUBRIC = """
ou must evaluate the **quality** of the feedback produced by an LLM.

Evaluate the feedback only using the following meta-rubric:

### META-RUBRIC

1. **Accuracy**
   Does the feedback correctly reflect the student’s submission?

2. **Specificity**
   Is the feedback concrete, detailed, and supported with examples rather than vague advice?

3. **Constructiveness**
   Does the feedback guide the student on how to improve?

4. **Alignment with assignment rubric**
   Does the feedback clearly reference (and stay aligned with) the criteria used to judge the work?

5. **Tone and clarity**
   Is the feedback easy to understand, supportive, and professionally written?

### SCORING INSTRUCTIONS
For each dimension, give a score from **0 to 10** and provide a justification.
Then provide an overall score and a summary.
"""


In [ ]:
# Define a sub-model for a single rubric criterion
class CriterionResult(BaseModel):
    score: int = Field(..., description="The score (0-10) for this dimension.")
    explanation: str = Field(..., description="The justification for the score.")

In [ ]:
# Define the main output model with explicit fields for each criterion
class JudgeOutput(BaseModel):
    accuracy: CriterionResult
    specificity: CriterionResult
    constructiveness: CriterionResult
    alignment: CriterionResult
    tone_clarity: CriterionResult

    overall_score: float = Field(
        ...,
        description="The average of the five scores, rounded to one decimal place."
    )
    summary: str = Field(
        ...,
        description="A short summary of the feedback's strengths and weaknesses."
    )

In [ ]:
# create the llm-as-judge evaluator

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
print("API Key set:", os.environ.get('OPENAI_API_KEY') is not None)

# judge = Agent(model="gpt-5-mini, retries=3")
judge = Agent(model="gpt-5-mini")

class MetaRubricEvaluator(Evaluator):

    async def evaluate(self, case):
        prompt = f"""
You are an expert academic evaluator.

The student's submission is:
--------------------
{student_submission}
--------------------

The feedback you must evaluate is:
--------------------
{llm_feedback}
--------------------

Use the following meta-rubric:
{META_RUBRIC}

Now score the feedback strictly according to the meta-rubric.
"""
        result = await judge.run(
            prompt,
            output_type=JudgeOutput # Pass the schema for structured output
        )
        # return EvaluationResult(model_output=result.output)
        return EvaluationResult(
            name="Meta_Rubric_Score", # Required field
            value=result.output.overall_score, # Extract the scalar score
            reason=result.output.summary, # Extract the summary as the reason
            source=judge.model.model_name
        )

API Key set: True


In [ ]:
# run the evaluation

evaluator = MetaRubricEvaluator()

case = {"id": "case1"}
result = await evaluator.evaluate(case)
result


EvaluationResult(name='Meta_Rubric_Score', value=8.4, reason="Strengths: The feedback is positive, aligned with the rubric, and offers a clear, actionable next step (investigate case studies and bypass techniques). It also scores the student's performance across appropriate categories and maintains a supportive tone. Weaknesses: The feedback occasionally mislabels the submission as 'prompts' instead of an essay, and it is light on concrete examples and specific resources or revision tasks. To improve, the educator should cite brief examples from the submission to justify each rubric score and recommend specific case studies or readings the student could examine.", source='gpt-5-mini')

In [ ]:
# # print full result
# print("\nFull Result Object:")
# print(result)

# pring score and reason
print(f"Score: {result.value}")
print("Reason:")
result.reason


Score: 8.4
Reason:


"Strengths: The feedback is positive, aligned with the rubric, and offers a clear, actionable next step (investigate case studies and bypass techniques). It also scores the student's performance across appropriate categories and maintains a supportive tone. Weaknesses: The feedback occasionally mislabels the submission as 'prompts' instead of an essay, and it is light on concrete examples and specific resources or revision tasks. To improve, the educator should cite brief examples from the submission to justify each rubric score and recommend specific case studies or readings the student could examine."